In [4]:
#Загружаю нужные библиотеки

import pandas as pd
import math
import re
import os

# для конвертации в словарь
import ast

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [13]:
file_path = 'C:/00_Data/03_stom/00_raw_data/med_aggrs_2021_12_dec/'

In [14]:
file_path_add = file_path + 'prodoctorov/'

In [16]:
# вытаскиваю названия файлов в список file_list

file_list = []

for root, dirs, files in os.walk(file_path_add):
    
    for filename in files:
        file_list += [filename]

In [17]:
file_list

['01_prodoctorov_msk.csv',
 '02_prodoctorov_spb.csv',
 '03_prodoctorov_nsk.csv',
 '04_prodoctorov_ekt.csv',
 '05_prodoctorov_kazan.csv',
 '06_prodoctorov_nnovgorod.csv',
 '07_prodoctorov_info_chelyabinsk.csv',
 '08_prodoctorov_info_samara.csv',
 '09_prodoctorov_info_omsk.csv',
 '10_prodoctorov_info_rostov-na-donu.csv',
 '11_prodoctorov_info_ufa.csv',
 '12_prodoctorov_info_krasnoyarsk.csv',
 '13_prodoctorov_info_voronezh.csv',
 '14_prodoctorov_info_perm.csv',
 '15_prodoctorov_info_volgograd.csv',
 '16_prodoctorov_info_krasnodar.csv',
 '17_prodoctorov_info_saratov.csv',
 '18_prodoctorov_info_tolyatti.csv',
 '19_prodoctorov_barnaul.csv',
 '20_prodoctorov_info_izhevsk.csv',
 '21_prodoctorov_info_ulyanovsk.csv',
 '22_prodoctorov_vladivostok.csv',
 '23_prodoctorov_info_yaroslavl.csv',
 '24_prodoctorov_info_irkutsk.csv',
 '25_prodoctorov_info_tyumen.csv',
 '26_prodoctorov_info_mahachkala.csv',
 '27_prodoctorov_info_habarovsk.csv',
 '28_prodoctorov_info_orenburg.csv']

In [33]:
work = pd.read_csv(file_path_add + file_list[0])

In [48]:
# все датасеты в один

work = pd.DataFrame()

for file in file_list:
    temp = pd.read_csv(file_path_add + file, low_memory=False)
    
    work = work.append(temp, ignore_index = True)
    
    work = work.drop_duplicates()

In [49]:
work.shape

(8021, 28)

In [63]:
%%time

# вытаскиваю данные из словаря в таблицу

data = [] # список для данных

for i in range(work.shape[0]):
    #ID_1dent = work.iloc[[i]]['ID_1dent'].values[0] 
    clinic = work.iloc[[i]]['clinic'].values[0]
    city = work.iloc[[i]]['city'].values[0]
    
    doctors_dict = ast.literal_eval(work.iloc[i].specialist)
    
    for doctor in doctors_dict:
        short_name = doctors_dict[doctor]['фио']
        full_name = doctors_dict[doctor]['имя']
        sex = doctors_dict[doctor]['пол']
        experience = doctors_dict[doctor]['стаж']
        specialization = doctors_dict[doctor]['профиль'].lower()
        review_number = doctors_dict[doctor]['кол-во отзывов']
        
        data.append([clinic, city,
                     short_name, full_name, sex,
                     experience, specialization, review_number])

Wall time: 5.97 s


In [64]:
len(data)

38488

In [65]:
head = ['clinic', 'city', 'short_name', 'full_name', 'sex', 'experience', 'specialization', 'review_number']
work_doctors = pd.DataFrame(data, columns=head)

In [66]:
work_doctors.sample(3)

,clinic,city,short_name,full_name,sex,experience,specialization,review_number
10001,Стоматология «Приорити»,Москва,Болдырева Е. С.,Болдырева Екатерина Сергеевна,F,18,стоматолог,1 отзыв
22013,«9 Стоматология» на Короленко,Казань,Шарафутдинова В. М.,Шарафутдинова Венера Муллаяровна,F,43,стоматолог-хирург,2 отзыва
32943,Стоматология «21 век»,Краснодар,Михайленко И. М.,Михайленко Иван Михайлович,M,18,стоматолог,9 отзывов


In [67]:
work_doctors.shape

(38488, 8)

In [68]:
work_doctors.dtypes

clinic            object
city              object
short_name        object
full_name         object
sex               object
experience        object
specialization    object
review_number     object
dtype: object

In [69]:
# проверка на пустые значения

work_doctors.isnull().sum()

clinic               0
city                 0
short_name           0
full_name            0
sex                  0
experience        5916
specialization       0
review_number        0
dtype: int64

In [70]:
work_doctors['review_number'] = work.review_number.replace(r'\s.+', '')

In [71]:
work_doctors.review_number.fillna('0', inplace=True)

In [72]:
work_doctors.experience.fillna('none', inplace=True)

In [77]:
# меняю типы столбцов

#work_doctors = work_doctors.astype({'experience': 'int32'})

In [86]:
work.dtypes

clinic                object
of_name               object
clinic_type           object
city                  object
address               object
metro_stations        object
work_hours            object
phone                 object
web_site              object
clinic_score          object
doc_score             object
doc_exp_score         object
review_number         object
good_reviews           int64
neutral_reviews        int64
bad_reviews            int64
clinic_features       object
related_clinics       object
doc_number            object
doc_number_2           int64
specialist            object
consultation_price    object
full_price            object
of_info               object
annotation            object
clinic_description    object
info_date             object
clinic_page           object
dtype: object

In [78]:
work.sample()

,clinic,of_name,clinic_type,city,address,metro_stations,work_hours,phone,web_site,clinic_score,...,doc_number,doc_number_2,specialist,consultation_price,full_price,of_info,annotation,clinic_description,info_date,clinic_page
7058,Стоматология «Унидент» на Пензенском бульваре,ООО «Унидент»,Стоматология,Ульяновск,"бульвар Пензенский, д. 22",{},"['ПН-ПТ: 09:00 — 20:00', 'СБ-ВС: 09:00 — 14:00']",none,http://www.unident-ul.ru,0.8,...,8,8,"{'257081': {'фио': 'Михайлова О. А.', 'имя': '...","{'Пародонтолог': 'от 120', 'Стоматолог': 'от 1...",{'Лечение зубов': {'Лечение кариеса': 'от 475'...,"{'address': 'бульвар Пензенский, д. 22', 'Вход...","Стоматология «Унидент» на Пензенском бульваре,...",Описание\nПроезд\nДо стоматологической клиники...,27.08.2020,https://prodoctorov.ru/ulyanovsk/lpu/32152-uni...


In [84]:
# количество клиник

(work.groupby('city', as_index=False)
 .agg({'clinic': 'count'})
 #.rename(columns={'divorce_number': 'average'})
 .sort_values('clinic', ascending=False)
).to_excel(file_path + 'temp.xlsx', index=False)

In [89]:
# количество врачей

(work.groupby('city', as_index=False)
 .agg({'clinic': 'count', 'doc_number_2': 'sum'})
 #.rename(columns={'divorce_number': 'average'})
 .sort_values('clinic', ascending=False)
)#.to_excel(file_path + 'temp.xlsx', index=False)

In [92]:
work_doctors.to_excel(file_path + 'temp.xlsx', index=False)

In [91]:
# количество врачей

(work_doctors.groupby(['full_name', 'clinic'], as_index=False)
 .agg({'clinic': 'count', 'doc_number_2': 'sum'})
 #.rename(columns={'divorce_number': 'average'})
 #.sort_values('clinic', ascending=False)
)#.to_excel(file_path + 'temp.xlsx', index=False)

### Средний стаж ###

In [75]:
# средний стаж врачей в клинике

doctor_avr_experience = (work_doctors[work_doctors.experience > 0]
                         .groupby(['clinic'], as_index=False)
                         .agg({'experience': 'mean'}).round(1)
                         .rename(columns={'experience': 'avr_experience'})
                        )

TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
doctor_avr_experience.sample(3)

In [ ]:
work = work.merge(doctor_avr_experience, how='left', on=['ID_1dent', 'clinic'])

In [ ]:
work.shape

### Специализация клиник ###

In [ ]:
# составляю список уникальных специальностей

doc_specialization = [] # список для данных

for i in work_doctors['specialization']: # вытаскиваю циклом
    if i != '': # fillna не помог
        doc_specialization += i.split(', ')
    
doc_specialization = list(set(doc_specialization)) # удаляю повторы

In [ ]:
# сортирую

doc_specialization.sort()

doc_specialization

In [ ]:

doc_specialization_dict = {} # словарь для специализаций

for specialty in doc_specialization:
    doc_number = 0
    
    for item in work_doctors['specialization'].values:
        for i in item.split(', '):
            doc_number += i.count(specialty)
                    
    doc_specialization_dict[specialty] = doc_number

In [ ]:
doc_specialization_dict

In [ ]:
header = ['specialty', 'doc_number']
work_doc_specialization = pd.DataFrame(list(doc_specialization_dict.items()), columns=header)

In [ ]:
work_doc_specialization

### Вытаскиваю специализации врачей в плоскую таблицу ###

In [ ]:
# вытаскиваю специализации 

for specialty in doc_specialization:
    temp_list = []
    
    for i in range(work_doctors.shape[0]):
        temp_list.append(work_doctors.loc[i].specialization.split(', ').count(specialty))
        
    work_doctors[specialty] = temp_list

In [ ]:
work_doctors.head()

In [ ]:
work_doctors.to_excel('C:/00_Data/Стоматологии/Clinic_Denta/temp.xlsx')

In [ ]:
doc_specialization

In [ ]:
# список с нужными специальностями

doc_specialization_need = ['гнатолог',
                           'детский стоматолог',
                           'пародонтолог',
                           'стоматолог',
                           'стоматолог-гигиенист',
                           'стоматолог-имплантолог',
                           'стоматолог-ортодонт',
                           'стоматолог-ортопед',
                           'стоматолог-хирург',
                           'терапевт',
                           'челюстно-лицевой хирург']

In [ ]:
clinic_specialization = (work_doctors
                         .groupby('clinic', as_index=False)
                         .agg({'гнатолог': 'sum',
                               'детский стоматолог': 'sum',
                               'пародонтолог': 'sum',
                               'стоматолог': 'sum',
                               'стоматолог-гигиенист': 'sum',
                               'стоматолог-имплантолог': 'sum',
                               'стоматолог-ортодонт': 'sum',
                               'стоматолог-ортопед': 'sum',
                               'стоматолог-хирург': 'sum',
                               'терапевт': 'sum',
                               'челюстно-лицевой хирург': 'sum'})
                        )

In [ ]:
clinic_specialization['all_specialties'] = clinic_specialization.iloc[:, 1:].sum(axis=1)

In [ ]:
clinic_specialization.sample()

In [ ]:
clinic_specialization['pediatric_%'] = round(
    clinic_specialization['детский стоматолог'] / clinic_specialization['all_specialties'],
    2)

In [ ]:
clinic_specialization['orthodontist_%'] = round(
    clinic_specialization['стоматолог-ортодонт'] / clinic_specialization['all_specialties'],
    2)

In [ ]:
work.sample()

In [ ]:
work = work.merge(clinic_specialization, on='clinic', how='left')

In [ ]:
work.to_excel('C:/00_Data/Стоматологии/Clinic_Denta/temp1.xlsx')